In [38]:
import pandas as pd
from sqlalchemy import create_engine
import pyarrow as pa
import pyarrow.parquet as pq
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import time

def scrape_website(url, pages=20):
    driver = None  # Inicializar driver como None

    try:
        options = Options()
        options.headless = True  # Cambiar a False para ver la interfaz gráfica de Chrome
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-popup-blocking")
        
        service = Service(r'C:\Users\cabg1\chromedriver-win64\chromedriver-win64\chromedriver.exe')
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(url)
        
        # Manejar la ventana emergente de cookies si es necesario
        try:
            accept_cookies_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.sc-bdVaJa.ebNrSm.sc-htoDjs.brhAsq.Button-bepvgg-0.dqiWxy.text-center.btn-disclaimer.btn.btn-secondary'))
            )
            accept_cookies_button.click()
            print('Se hizo clic en el botón de aceptar cookies')
        except Exception as e:
            print(f'No se pudo aceptar cookies: {str(e)}')
        
        titles, descriptions = [], []

        for page in range(1, pages + 1):
            print(f"Scraping página {page}...")

            # Esperar a que los elementos de precio y título estén presentes
            WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-fMiknA')))
            WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-dxgOiQ')))
            
            # Obtener el contenido HTML después de la carga completa
            page_source = driver.page_source
            
            # Parsear el contenido HTML con BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')
            
            # Encontrar todos los elementos de título
            title_elements = soup.find_all('h2', class_='sc-dxgOiQ BSoGx card-title')
            
            # Encontrar todos los elementos de descripción
            description_elements = soup.find_all('ul', class_='sc-iRbamj kYQRXi inline-list-grid')
            
            # Extraer y organizar los datos
            for title_element in title_elements:
                titles.append(title_element.div.text.strip())

            for description_element in description_elements:
                description_text = " | ".join([item.text.strip() for item in description_element.find_all('p', class_='sc-fMiknA ZUMHA card-subitem text-black')])
                descriptions.append(description_text)
            
            # Hacer clic en el botón de siguiente página si no es la última página
            if page < pages:
                try:
                    next_page_number = str(page + 1)
                    next_page_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.LINK_TEXT, next_page_number))
                    )
                    next_page_button.click()
                    time.sleep(3)  # Esperar unos segundos para que la página cargue
                except Exception as e:
                    print(f'No se pudo pasar a la siguiente página: {str(e)}')
                    break

        # Asegurarse de que todas las listas tengan la misma longitud
        min_length = min(len(titles), len(descriptions))
        titles = titles[:min_length]
        descriptions = descriptions[:min_length]

        # Obtener la fecha actual
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        dates = [current_date] * min_length
        
        # Crear un DataFrame
        df = pd.DataFrame({
            'Fecha de Scraping': dates,
            'Título': titles,
            'Descripción': descriptions
        })

        # Obtener la fecha actual y formatearla
        fecha_actual = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        # Ruta relativa dentro de tu repositorio local clonado
        ruta_archivo = f'C:/Users/cabg1/ScrapingMetroCuadrado/scraping/Arriendo_scraping_{fecha_actual}.csv'

        # Guardar el DataFrame como archivo CSV en el repositorio local
        df.to_csv(ruta_archivo, index=False)
        print(f'Archivo CSV guardado exitosamente en el repositorio: {ruta_archivo}')

        # Mostrar df.head() en otra línea
        print(df.head())
        
    except Exception as e:
        print(f'Error general: {str(e)}')

    finally:
        if driver is not None:  # Verificar si driver está definido
            driver.quit()

# Ejemplo de uso: scraping inicial de las 10 primeras páginas
scrape_website('https://www.metrocuadrado.com/arriendo/')

Se hizo clic en el botón de aceptar cookies
Scraping página 1...
Scraping página 2...
Scraping página 3...
Scraping página 4...
Scraping página 5...
Scraping página 6...
Scraping página 7...
Scraping página 8...
Scraping página 9...
Scraping página 10...
Scraping página 11...
Scraping página 12...
Scraping página 13...
Scraping página 14...
Scraping página 15...
Scraping página 16...
Scraping página 17...
Scraping página 18...
Scraping página 19...
Scraping página 20...
Archivo CSV guardado exitosamente en el repositorio: C:/Users/cabg1/ScrapingMetroCuadrado/scraping/Arriendo_scraping_2024-06-23_17-39-49.csv
     Fecha de Scraping                                       Título  \
0  2024-06-23 17:39:49      Casa en Arriendo, Santa Cecilia 5, Chía   
1  2024-06-23 17:39:49     Casa en Arriendo, Guaymaral, Bogotá D.C.   
2  2024-06-23 17:39:49  Casa en Arriendo, El Porvenir, Barranquilla   
3  2024-06-23 17:39:49      Casa en Arriendo, PROVENZA, Bucaramanga   
4  2024-06-23 17:39:49    Cas

In [39]:
import os

# Directorio donde se guardan los archivos CSV históricos (ajusta la ruta según tu estructura)
directorio_historico = 'C:/Users/cabg1/ScrapingMetroCuadrado/scraping/'

# Lista para almacenar DataFrames de todos los archivos históricos
dfs_historicos = []

# Leer archivos CSV históricos y almacenar en una lista de DataFrames
for archivo in os.listdir(directorio_historico):
    if archivo.endswith('.csv') and archivo.startswith('Arriendo_scraping'):
        ruta_archivo = os.path.join(directorio_historico, archivo)
        df = pd.read_csv(ruta_archivo)
        dfs_historicos.append(df)

# Concatenar todos los DataFrames en uno solo
df_historico_completo = pd.concat(dfs_historicos, ignore_index=True)

# Mostrar información básica del DataFrame histórico completo
print("Información del DataFrame histórico completo:")
print(df_historico_completo.info())


Información del DataFrame histórico completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Fecha de Scraping  11000 non-null  object
 1   Título             11000 non-null  object
 2   Descripción        7407 non-null   object
dtypes: object(3)
memory usage: 257.9+ KB
None


In [42]:
# Crear una clave única basada en las características clave del inmueble
df_historico_completo['Clave_Unica'] = (
    df_historico_completo['Fecha de Scraping'].astype(str) + '_' +
    df_historico_completo['Título'].astype(str) + '_' +
    df_historico_completo['Descripción'].astype(str)
)

In [43]:
df = df_historico_completo[['Clave_Unica', 'Fecha de Scraping', 'Título', 'Descripción']].copy()
df

,Clave_Unica,Fecha de Scraping,Título,Descripción
0,"2024-06-23 16:06:43_Casa en Arriendo, Santa Ce...",2024-06-23 16:06:43,"Casa en Arriendo, Santa Cecilia 5, Chía",60 | 2 | 2
1,"2024-06-23 16:06:43_Casa en Arriendo, Guaymara...",2024-06-23 16:06:43,"Casa en Arriendo, Guaymaral, Bogotá D.C.",190 | 3 | 5
2,"2024-06-23 16:06:43_Casa en Arriendo, El Porve...",2024-06-23 16:06:43,"Casa en Arriendo, El Porvenir, Barranquilla",100 | 4 | 3
3,"2024-06-23 16:06:43_Casa en Arriendo, PROVENZA...",2024-06-23 16:06:43,"Casa en Arriendo, PROVENZA, Bucaramanga",$6.300.000 | 295 m² | 3 | 5
4,"2024-06-23 16:06:43_Casa en Arriendo, La Cumbr...",2024-06-23 16:06:43,"Casa en Arriendo, La Cumbre, Barranquilla",$6.300.000 | 295 m² | 3 | 5
...,...,...,...,...
10995,"2024-06-23 17:39:49_Apartamento en Arriendo, Z...",2024-06-23 17:39:49,"Apartamento en Arriendo, Zona Sur, Cali",NaN
10996,"2024-06-23 17:39:49_Casa en Arriendo, Loma Del...",2024-06-23 17:39:49,"Casa en Arriendo, Loma Del Atravesado, Envigado",$2.500.000 | 100 m² | 3 | 4
10997,"2024-06-23 17:39:49_Apartamento en Arriendo, C...",2024-06-23 17:39:49,"Apartamento en Arriendo, Castropol, Medellín",$2.500.000 | 100 m² | 3 | 4
10998,"2024-06-23 17:39:49_Apartamento en Arriendo, C...",2024-06-23 17:39:49,"Apartamento en Arriendo, Carvajal Osorio, Bogo...",NaN


In [44]:
num_filas = df.shape[0]
print(f'El DataFrame tiene {num_filas} filas.')


El DataFrame tiene 11000 filas.


In [45]:
df['Fecha de Scraping'] = pd.to_datetime(df['Fecha de Scraping'], format='%Y-%m-%d %H:%M:%S')

In [47]:
registros_iniciales = len(df)

df = df.drop_duplicates(subset='Clave_Unica', keep='first')

registros_eliminados = registros_iniciales - len(df)


In [48]:
print(f'Se eliminaron {registros_eliminados} registros duplicados de la fecha de scraping más reciente.')


Se eliminaron 1599 registros duplicados de la fecha de scraping más reciente.


In [49]:
# Eliminar filas con NaN en la columna 'descripcion'
df = df.dropna(subset=['Descripción'])
df.head()

,Clave_Unica,Fecha de Scraping,Título,Descripción
0,"2024-06-23 16:06:43_Casa en Arriendo, Santa Ce...",2024-06-23 16:06:43,"Casa en Arriendo, Santa Cecilia 5, Chía",60 | 2 | 2
1,"2024-06-23 16:06:43_Casa en Arriendo, Guaymara...",2024-06-23 16:06:43,"Casa en Arriendo, Guaymaral, Bogotá D.C.",190 | 3 | 5
2,"2024-06-23 16:06:43_Casa en Arriendo, El Porve...",2024-06-23 16:06:43,"Casa en Arriendo, El Porvenir, Barranquilla",100 | 4 | 3
3,"2024-06-23 16:06:43_Casa en Arriendo, PROVENZA...",2024-06-23 16:06:43,"Casa en Arriendo, PROVENZA, Bucaramanga",$6.300.000 | 295 m² | 3 | 5
4,"2024-06-23 16:06:43_Casa en Arriendo, La Cumbr...",2024-06-23 16:06:43,"Casa en Arriendo, La Cumbre, Barranquilla",$6.300.000 | 295 m² | 3 | 5


In [50]:
# Separar la columna 'Título' en nuevas columnas: Tipo de Inmueble, Sector, Ciudad
df[['Tipo de Inmueble', 'Sector', 'Ciudad']] = df['Título'].str.split(',', expand=True)

# Separar la columna 'Descripción' en nuevas columnas: Precio, Superficie, Habitaciones, Baños
df[['Precio', 'Superficie', 'Habitaciones', 'Baños']] = df['Descripción'].str.split('|', expand=True)

# Limpieza adicional para eliminar espacios en blanco alrededor de cada valor
df['Tipo de Inmueble'] = df['Tipo de Inmueble'].str.strip()
df['Sector'] = df['Sector'].str.strip()
df['Ciudad'] = df['Ciudad'].str.strip()
df['Precio'] = df['Precio'].str.strip()
df['Superficie'] = df['Superficie'].str.strip()
df['Habitaciones'] = df['Habitaciones'].str.strip()
df['Baños'] = df['Baños'].str.strip()

In [51]:
# Crear df1 eliminando las filas donde 'Baños' contiene NaN o None
df = df.dropna(subset=['Baños'])

In [52]:
# Crear df1 eliminando las columnas 'Título' y 'Descripción'
df1 = df.drop(['Título', 'Descripción'], axis=1)

# Mostrar el DataFrame resultante
print(df1)

                                             Clave_Unica   Fecha de Scraping  \
3      2024-06-23 16:06:43_Casa en Arriendo, PROVENZA... 2024-06-23 16:06:43   
4      2024-06-23 16:06:43_Casa en Arriendo, La Cumbr... 2024-06-23 16:06:43   
6      2024-06-23 16:06:43_Casa en Arriendo, ALTOS DE... 2024-06-23 16:06:43   
7      2024-06-23 16:06:43_Apartamento en Arriendo, A... 2024-06-23 16:06:43   
9      2024-06-23 16:06:43_Casa en Arriendo, SANTA BA... 2024-06-23 16:06:43   
...                                                  ...                 ...   
10993  2024-06-23 17:39:49_Apartamento en Arriendo, C... 2024-06-23 17:39:49   
10994  2024-06-23 17:39:49_Apartamento en Arriendo, L... 2024-06-23 17:39:49   
10996  2024-06-23 17:39:49_Casa en Arriendo, Loma Del... 2024-06-23 17:39:49   
10997  2024-06-23 17:39:49_Apartamento en Arriendo, C... 2024-06-23 17:39:49   
10999  2024-06-23 17:39:49_Casa en Arriendo, La Ceja,... 2024-06-23 17:39:49   

              Tipo de Inmueble         

In [53]:
# Contar registros por ciudad y obtener el top 10
top_20_ciudades = df1['Ciudad'].value_counts().head(20)

# Mostrar los resultados
print(top_20_ciudades)

Ciudad
Bogotá D.C.            2008
Medellín                890
Envigado                351
Cali                    197
Barranquilla            191
Sabaneta                154
Manizales               146
Pereira                 104
Chía                     89
Rionegro                 81
Cajicá                   57
Cartagena de Indias      57
Retiro                   47
Jamundi                  39
Itagui                   34
Puerto Colombia          32
Bello                    28
Bucaramanga              25
La Calera                25
Santa Marta              25
Name: count, dtype: int64


In [54]:
# Ciudades especificadas
ciudades_especificadas = ['Bogotá D.C.', 'Medellín', 'Envigado', 'Cali', 'Barranquilla', 'Sabaneta',
                          'Manizales', 'Pereira', 'Chía', 'Rionegro', 'Cartagena de Indias']

# Filtrar el DataFrame original para incluir solo las ciudades especificadas
df_filter = df1[df1['Ciudad'].isin(ciudades_especificadas)]

# Mostrar el DataFrame filtrado
print(df_filter)

                                             Clave_Unica   Fecha de Scraping  \
4      2024-06-23 16:06:43_Casa en Arriendo, La Cumbr... 2024-06-23 16:06:43   
6      2024-06-23 16:06:43_Casa en Arriendo, ALTOS DE... 2024-06-23 16:06:43   
9      2024-06-23 16:06:43_Casa en Arriendo, SANTA BA... 2024-06-23 16:06:43   
10     2024-06-23 16:06:43_Casa en Arriendo, ENVIGADO... 2024-06-23 16:06:43   
12     2024-06-23 16:06:43_Casa en Arriendo, San Luca... 2024-06-23 16:06:43   
...                                                  ...                 ...   
10991  2024-06-23 17:39:49_Consultorio en Arriendo, C... 2024-06-23 17:39:49   
10993  2024-06-23 17:39:49_Apartamento en Arriendo, C... 2024-06-23 17:39:49   
10994  2024-06-23 17:39:49_Apartamento en Arriendo, L... 2024-06-23 17:39:49   
10996  2024-06-23 17:39:49_Casa en Arriendo, Loma Del... 2024-06-23 17:39:49   
10997  2024-06-23 17:39:49_Apartamento en Arriendo, C... 2024-06-23 17:39:49   

              Tipo de Inmueble         

In [56]:
df_filter.dtypes

Clave_Unica                  object
Fecha de Scraping    datetime64[ns]
Tipo de Inmueble             object
Sector                       object
Ciudad                       object
Precio                       object
Superficie                   object
Habitaciones                 object
Baños                        object
dtype: object